In [1]:
from __future__  import annotations
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, cycle, product, islice, chain
from functools   import lru_cache
from typing      import Dict, Tuple, Set, List, Iterator, Optional
from sys         import maxsize

import re
import ast
import operator

import numpy as np

In [2]:
def read_data(input: str, parser=str, sep='\n', testing=False) -> list:
    if testing:
        sections = input.split(sep)
    else:
        sections = open(input).read().split(sep)
    return [parser(section) for section in sections]

In [3]:
test_string = """0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b"

ababbb
bababa
abbbab
aaabbb
aaaabbb"""
test_ins = read_data(test_string, sep="\n\n", testing=True)

In [4]:
def parse_rule(string):
    key, val = re.match(r"(\d+): (.+)", string.replace('"', '')).groups()
    if len(val) == 1:
        return int(key), val
    # get 4 5 | 5 4 into the form ([4,5], [5,4])
    # and 3 5 6 into the form [3, 5, 6]
    # a tuple represents options, a list represents a seq of rules
    elif "|" in val:
        vals = val.split(" | ")
        res = tuple(list(map(int, val.split(" "))) for val in vals)
    else:
        res = list(list(map(int, val.split(" "))))
    return int(key), res

def parse_rules(strings):
    raw_rules = strings.split("\n")
    return dict(parse_rule(rule) for rule in raw_rules)

def calculate_match(string: str, rule, rules: dict) -> int:
    '''return 1 if match between string and rule'''
    if not string and not rule:
        # previous part of string exactly matched
        return 1
    if not string or not rule:
        # extra in string or 
        return 0
    if string[0] == rule[0]:
        return calculate_match(string[1:], rule[1:], rules)
    if isinstance(rule[0], int):
        new_rule = rules[rule[0]]
        return calculate_match(string, (rules[rule[0]], *rule[1:]), rules) 
    if isinstance(rule[0], list):
        # if first element in rule is a list of rules
        curr_rule = rule[0]
        return calculate_match(string, (*curr_rule, *rule[1:]), rules)
    if isinstance(rule[0], tuple):
        # if first element in rule is a tuple of rule options
        curr_rule = rule[0]
        for option in curr_rule:
            res = calculate_match(string, (option, *rule[1:]), rules)
            if res:
                return res
    return 0

def run_part1(input: List[str]):
    raw_rules, raw_strings = input
    rules = parse_rules(raw_rules)
    strings = raw_strings.split()
    return sum(calculate_match(string, rules[0], rules) for string in strings)

In [5]:
run_part1(test_ins) 

2

Part I  

How many messages completely match rule 0?

In [6]:
real_ins = read_data("input.txt", sep="\n\n")
run_part1(real_ins)

216

Part II

After updating rules 8 and 11, how many messages completely match rule 0?

In [7]:
def run_part2(input: List[str]):
    raw_rules, raw_strings = input
    # replace the rules manually such that the rule on the left always goes first
    # if it suceeds, then there's no need to further recurse, avoiding infinite recursion
    rules = {**parse_rules(raw_rules), 8: [42, ([], [8])], 11: [42, ([], [11]), 31]}
    strings = raw_strings.split()
    return sum(calculate_match(string, rules[0], rules) for string in strings)

In [8]:
test_string2 = """42: 9 14 | 10 1
9: 14 27 | 1 26
10: 23 14 | 28 1
1: "a"
11: 42 31
5: 1 14 | 15 1
19: 14 1 | 14 14
12: 24 14 | 19 1
16: 15 1 | 14 14
31: 14 17 | 1 13
6: 14 14 | 1 14
2: 1 24 | 14 4
0: 8 11
13: 14 3 | 1 12
15: 1 | 14
17: 14 2 | 1 7
23: 25 1 | 22 14
28: 16 1
4: 1 1
20: 14 14 | 1 15
3: 5 14 | 16 1
27: 1 6 | 14 18
14: "b"
21: 14 1 | 1 14
25: 1 1 | 1 14
22: 14 14
8: 42
26: 14 22 | 1 20
18: 15 15
7: 14 5 | 1 21
24: 14 1

abbbbbabbbaaaababbaabbbbabababbbabbbbbbabaaaa
bbabbbbaabaabba
babbbbaabbbbbabbbbbbaabaaabaaa
aaabbbbbbaaaabaababaabababbabaaabbababababaaa
bbbbbbbaaaabbbbaaabbabaaa
bbbababbbbaaaaaaaabbababaaababaabab
ababaaaaaabaaab
ababaaaaabbbaba
baabbaaaabbaaaababbaababb
abbbbabbbbaaaababbbbbbaaaababb
aaaaabbaabaaaaababaa
aaaabbaaaabbaaa
aaaabbaabbaaaaaaabbbabbbaaabbaabaaa
babaaabbbaaabaababbaabababaaab
aabbbbbaabbbaaaaaabbbbbababaaaaabbaaabba
"""
test_ins2 = read_data(test_string2, sep="\n\n", testing=True)

In [9]:
run_part2(test_ins2)

12

In [10]:
run_part2(real_ins)

400